# Хакатон: "Hack the cart 2022"

Организатор - Лента.
Ссылка: https://hackthecart.ru/
Ссылка на памятку: https://russianhackers.notion.site/Hack-The-Cart-b7670766613947cca3b13b3e6d924ad7

hist_data.csv - исторические данные о покупках
- buyer_id - id покупателя
- pav_order_id - id заказа
- created - время добавления в заказ
- item_id - id item'а
- count - количество выбранного товара в заказе
- price_sold - цена за 1 item
- flag_weight_goods - бинарный флаг того, является ли товар весовым
- weight - вес заказа

test.csv - текущее состояние корзины (последний заказ пользователя)
- buyer_id - id покупателя
- pav_order_id - id заказа
- created - время добавления в заказ
- item_id - id item'а (товара)
- count - количество выбранного товара в заказе
- price_sold - цена за 1 item
- flag_weight_goods - бинарный флаг того, является ли товар весовым

In [31]:
import pandas as pd
import numpy as np
import gc

from collections import Counter 

# настройка опций отображения
pd.options.display.float_format = '{:.1f}'.format
pd.set_option('display.max_columns', None)

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

In [32]:
hist_data = pd.read_csv('hist_data.csv')

In [33]:
# добавляем день, месяц, год
hist_data['year'] = pd.DatetimeIndex(hist_data['created']).year
hist_data['month'] = pd.DatetimeIndex(hist_data['created']).month
hist_data['day'] = pd.DatetimeIndex(hist_data['created']).day

In [34]:
# добавляем итог по купленному товару
hist_data['total_price'] = hist_data['count'] * hist_data['price_sold']

In [35]:
hist_data

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight,year,month,day,total_price
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.0,80.0,False,11.1,2021,7,1,80.0
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.1,44.9,True,11.1,2021,7,1,48.2
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.0,70.0,False,11.1,2021,7,1,70.0
3,95203091,98506637863,2021-07-01 00:03:44,202820143,2.0,41.3,True,11.1,2021,7,1,81.4
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.0,270.0,False,11.1,2021,7,1,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,2021-09-07 18:48:29,204351817,1.0,100.0,False,9.8,2021,9,7,100.0
4529885,95619205,4620221347,2021-09-07 18:48:29,203523604,1.0,62.2,False,9.8,2021,9,7,62.2
4529886,95619205,4620221347,2021-09-07 18:48:29,203566357,2.0,39.9,False,9.8,2021,9,7,79.8
4529887,95619205,4620221347,2021-09-07 18:48:29,203566418,1.0,62.2,False,9.8,2021,9,7,62.2


In [36]:
hist_data[hist_data['buyer_id'] == 96400990]

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight,year,month,day,total_price
4529301,96400990,5700017986,2021-09-07 10:00:40,202808690,1.0,246.5,False,12.5,2021,9,7,246.5
4529302,96400990,5700017986,2021-09-07 10:00:40,203445463,1.0,200.0,False,12.5,2021,9,7,200.0
4529303,96400990,5700017986,2021-09-07 10:00:40,211101237,1.0,69.9,False,12.5,2021,9,7,69.9
4529304,96400990,5700017986,2021-09-07 10:00:40,202927858,1.0,133.0,False,12.5,2021,9,7,133.0
4529305,96400990,5700017986,2021-09-07 10:00:40,203438084,1.0,182.0,False,12.5,2021,9,7,182.0
4529306,96400990,5700017986,2021-09-07 10:00:40,203457302,1.0,110.0,False,12.5,2021,9,7,110.0
4529307,96400990,5700017986,2021-09-07 10:00:40,203029021,2.7,567.0,True,12.5,2021,9,7,1530.8
4529308,96400990,5700017986,2021-09-07 10:00:40,203029022,0.9,330.7,True,12.5,2021,9,7,297.7
4529309,96400990,5700017986,2021-09-07 10:00:40,202793061,3.0,57.0,False,12.5,2021,9,7,171.0
4529310,96400990,5700017986,2021-09-07 10:00:40,202997964,1.0,33.0,False,12.5,2021,9,7,33.0


In [37]:
hist_data[hist_data['buyer_id'] == 96400990].groupby(['buyer_id'])['count'].count()

buyer_id
96400990    12
Name: count, dtype: int64

In [38]:
#hist_data.info()

In [39]:
#hist_data.head()

In [40]:
#hist_data.describe()

In [41]:
#hist_data['item_id'].nlargest(10)

### Создание нового датасета с характеристиками покупателей

In [42]:
# создаём новую переменную
df = pd.DataFrame()

# отбираем уникальные id покупателей и подсчитываем количество купленных товаров за всё время
df = hist_data.groupby('buyer_id')['pav_order_id'].count().reset_index()

# добавляем количество заказов клиента
orders_count = hist_data.groupby('buyer_id')['pav_order_id'].apply(lambda x: x.unique().size).sort_values(ascending=False)\
                        .reset_index()
orders_count.rename(columns={'pav_order_id': 'orders_count'}, inplace=True)
df = df.merge(orders_count, how='left' , on='buyer_id')

# находим дату первой покупки и добавляем к датасету
min_purs_date = hist_data.groupby(['buyer_id'])['created'].min().reset_index()
df = df.merge(min_purs_date, how='left' , on='buyer_id')
df.rename(columns={'pav_order_id': 'items_count', 'created': 'first_purchases'}, inplace=True)


# определяем дату последней покупки
max_purs_date = hist_data.groupby(['buyer_id'])['created'].max().reset_index()
df = df.merge(max_purs_date, how='left' , on='buyer_id')
df.rename(columns={'created': 'last_purchases'}, inplace=True)

# добавляем сумму всех покупок
sum_check = hist_data.groupby(['buyer_id'])['total_price'].sum().reset_index()
df = df.merge(sum_check, how='left' , on='buyer_id')
df.rename(columns={'total_price': 'sum_check'}, inplace=True)

# расчитываем средний чек
#average_check = hist_data.groupby(['buyer_id'])['total_price'].mean().reset_index()
#df = df.merge(average_check, how='left' , on='buyer_id')
#df.rename(columns={'total_price': 'average_check'}, inplace=True)
df['avg_check'] = df['sum_check'] / df['orders_count']


# подсчитываем среднее количество купленных товаров и добавляем к датасету
#purshase = hist_data.groupby(['buyer_id', 'pav_order_id'])['item_id'].count().reset_index()
#purshase_mean = purshase.groupby(['buyer_id'])['item_id'].mean().reset_index()
#df = df.merge(purshase_mean, how='left' , on='buyer_id')
#df.rename(columns={'item_id': 'purchases_mean'}, inplace=True)
df['purchases_count_mean'] = df['items_count'] / df['orders_count']


# рассчитываем дисперсию
df['dispersion'] = df['purchases_count_mean'].std() / df['purchases_count_mean']

df

,buyer_id,items_count,orders_count,first_purchases,last_purchases,sum_check,avg_check,purchases_count_mean,dispersion
0,94578442,37,3,2021-07-02 17:20:51,2021-08-08 09:58:03,5652.1,1884.0,12.3,0.6
1,94578475,38,2,2021-07-20 19:50:41,2021-08-07 15:31:12,5416.9,2708.5,19.0,0.4
2,94578533,62,4,2021-08-02 12:20:15,2021-08-13 12:48:43,6499.3,1624.8,15.5,0.5
3,94578595,43,3,2021-07-12 11:01:33,2021-08-17 12:19:39,9316.7,3105.6,14.3,0.5
4,94578656,80,6,2021-07-06 16:32:22,2021-08-07 20:01:09,7511.9,1252.0,13.3,0.6
...,...,...,...,...,...,...,...,...,...
63920,96395136,15,1,2021-09-06 12:57:18,2021-09-06 12:57:18,2553.5,2553.5,15.0,0.5
63921,96397556,47,1,2021-09-06 18:26:01,2021-09-06 18:26:01,3385.5,3385.5,47.0,0.2
63922,96399270,16,1,2021-09-06 22:25:13,2021-09-06 22:25:13,1314.0,1314.0,16.0,0.5
63923,96399769,14,1,2021-09-07 03:45:30,2021-09-07 03:45:30,1700.4,1700.4,14.0,0.5


In [43]:
df[df['buyer_id'] == 94740780]

,buyer_id,items_count,orders_count,first_purchases,last_purchases,sum_check,avg_check,purchases_count_mean,dispersion
8590,94740780,716,10,2021-07-03 13:38:57,2021-09-05 13:23:44,89451.0,8945.1,71.6,0.1


In [44]:
df2 = hist_data[['buyer_id', 'pav_order_id', 'created']]
df2.drop_duplicates(subset=['buyer_id', 'pav_order_id', 'created'], inplace=True)
df2['date'] = pd.to_datetime(df2['created']).dt.date
df2['pav_marker'] = 1
df2

,buyer_id,pav_order_id,created,date,pav_marker
0,95203091,98506637863,2021-07-01 00:03:44,2021-07-01,1
16,95153036,98506638027,2021-07-01 00:05:44,2021-07-01,1
32,95854621,98506638756,2021-07-01 00:16:39,2021-07-01,1
70,94767798,98506638757,2021-07-01 00:16:53,2021-07-01,1
123,94748662,98506638761,2021-07-01 00:17:38,2021-07-01,1
...,...,...,...,...,...
4529783,94923960,98521217318,2021-09-07 16:17:36,2021-09-07,1
4529810,95619205,4620221265,2021-09-07 17:41:47,2021-09-07,1
4529824,95619205,4620221287,2021-09-07 17:54:27,2021-09-07,1
4529843,94806224,98521253887,2021-09-07 18:38:15,2021-09-07,1


In [45]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231339 entries, 0 to 4529864
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   buyer_id      231339 non-null  int64 
 1   pav_order_id  231339 non-null  int64 
 2   created       231339 non-null  object
 3   date          231339 non-null  object
 4   pav_marker    231339 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 10.6+ MB


In [47]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [48]:
# как часто покупал товары клиент за последниe 7 дней от даты последней покупки
df2['freqensy_7d']=df2.groupby('buyer_id').parallel_apply(lambda x : x.set_index('date').rolling(window=7).count())\
                                ['pav_order_id'].values#.reset_index() #closed='right' #, center=False , min_periods=1, closed='left'
#'pav_marker'

In [49]:
df2.sort_values(by=['buyer_id' , 'pav_order_id' , 'created'], ascending=False)

,buyer_id,pav_order_id,created,date,pav_marker,freqensy_7d
4529301,96400990,5700017986,2021-09-07 10:00:40,2021-09-07,1,1.0
4529029,96399769,98521073275,2021-09-07 03:45:30,2021-09-07,1,1.0
4529001,96399270,98521059586,2021-09-06 22:25:13,2021-09-06,1,1.0
4528876,96397556,98521022564,2021-09-06 18:26:01,2021-09-06,1,1.0
4528174,96395136,5700017512,2021-09-06 12:57:18,2021-09-06,1,1.0
...,...,...,...,...,...,...
2817804,94578475,98514370660,2021-08-07 15:31:12,2021-08-07,1,3.0
1491174,94578475,98510926135,2021-07-20 19:50:41,2021-07-20,1,1.0
2856604,94578442,98514485737,2021-08-08 09:58:03,2021-08-08,1,2.0
1111459,94578442,98509860323,2021-07-15 17:37:29,2021-07-15,1,2.0


In [50]:
df2[df2['buyer_id'] == 94578442] 

,buyer_id,pav_order_id,created,date,pav_marker,freqensy_7d
137478,94578442,98507010775,2021-07-02 17:20:51,2021-07-02,1,2.0
1111459,94578442,98509860323,2021-07-15 17:37:29,2021-07-15,1,2.0
2856604,94578442,98514485737,2021-08-08 09:58:03,2021-08-08,1,2.0


Датасет потребительской корзины

In [51]:
basket = pd.DataFrame

In [52]:
basket = hist_data.groupby(['pav_order_id'])['item_id'].count().reset_index()
mean_item_id = hist_data.groupby(['pav_order_id'])['item_id'].mean()
basket.merge(mean_item_id, on='pav_order_id')
basket

,pav_order_id,item_id
0,4620121471,29
1,4620121478,22
2,4620121484,14
3,4620121490,20
4,4620121510,21
...,...,...
231334,98521174139,14
231335,98521186483,61
231336,98521189556,12
231337,98521217318,27


In [53]:
# датасет покупок пользователей, в разрезе товаров, дат потребления
df_buyers = pd.DataFrame
df_buyers = hist_data.groupby(['buyer_id', 'item_id', 'created'])[['pav_order_id']].count().reset_index()
df_buyers.head(10)

,buyer_id,item_id,created,pav_order_id
0,94578442,202795926,2021-08-08 09:58:03,1
1,94578442,202801709,2021-07-15 17:37:29,1
2,94578442,202801709,2021-08-08 09:58:03,1
3,94578442,202806979,2021-07-02 17:20:51,1
4,94578442,202806979,2021-07-15 17:37:29,1
5,94578442,202806998,2021-07-15 17:37:29,1
6,94578442,202807303,2021-07-02 17:20:51,1
7,94578442,202808803,2021-08-08 09:58:03,1
8,94578442,202819114,2021-07-02 17:20:51,1
9,94578442,202820148,2021-07-02 17:20:51,1


На примере покупателя 94578442, он приобретал одинаковые товары 202801709 и 202806979 в разное время.

In [54]:
df_buyers[df_buyers['pav_order_id']>1]

,buyer_id,item_id,created,pav_order_id


In [55]:
df_buyers_items = hist_data.groupby(['buyer_id', 'item_id'])['pav_order_id'].count().reset_index()
df_buyers_items

,buyer_id,item_id,pav_order_id
0,94578442,202795926,1
1,94578442,202801709,2
2,94578442,202806979,2
3,94578442,202806998,1
4,94578442,202807303,1
...,...,...,...
3644161,96400990,203445463,1
3644162,96400990,203457302,1
3644163,96400990,203581478,1
3644164,96400990,205797303,1


In [ ]:
# количество покупок каждого товара. т.о. получаем ДС с самыми популярными товарами

items_sales_count = hist_data.groupby(['item_id'])[['pav_order_id']].count().sort_values(by='pav_order_id', ascending=False)\
                             .reset_index()
items_sales_count.rename(columns={'pav_order_id': 'count'}, inplace=True)

items_sales_count

,item_id,count
0,202820148,65840
1,202872237,41658
2,202809628,27245
3,202838754,17863
4,202862432,17640
...,...,...
43776,203197809,2
43777,204060930,2
43778,211062761,2
43779,204364616,2


In [ ]:
# определяем ТОП-50 товаров, которые чаще всего приобретают

top_50_items = items_sales_count.nlargest(50, 'count')
top_50_items.head(5)

,item_id,count
0,202820148,65840
1,202872237,41658
2,202809628,27245
3,202838754,17863
4,202862432,17640
